# DEPENDENCIAS

Puedes ejecutar una vez este codigo y luego borrarlo.

# Librerias

In [7]:
#import requests
import pandas as pd
import re
import json
import os
from bs4 import BeautifulSoup as bs

import random

import time


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

import undetected_chromedriver as uc

from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel

# Entradas

In [8]:
# Ruta local al driver de Chrome
ruta_driver = r"chromedriver-win64/chromedriver.exe"

# Configuración de opciones del navegador
chrome_options = Options()
# chrome_options.add_argument('--headless')  # Ejecutar sin abrir ventana
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Obtención de Enlaces

In [11]:
# Abre Google
service = Service(executable_path=ruta_driver)
browser = webdriver.Chrome(service=service, options=chrome_options)

# Esta variable solo servira para asignarle nombre al Excel que se generara al final.
# Te recomiendo ponerle algo con lo que lo diferencies de los demás.
busqueda = 'Software Architect'

# AQUI poner el enlace de GLASSDOOR con el trabajo ya buscado:
# Abres ESTE mismo enlace
# Buscas el trabajo que necesitas "Scrappear"
# Copias el enlace principal y lo pegas acá.
url = 'https://www.glassdoor.com.mx/Empleo/estados-unidos-software-engineer-empleos-SRCH_IL.0,14_IN1_KO15,32.html'

print(url)

# Se ingresa el URL asignado
browser.get(url)

# Obtención de estructura HTML
html = browser.page_source
soup = bs(html, 'lxml')

# Filtrado de enlaces
articles = soup.find('ul', {'class': 'JobsList_jobsList__lqjTr'}).find_all('a', {'class': 'JobCard_jobTitle__GLyJ1'})
link_chambas = [article.get('href') for article in articles]

browser.quit()

# Selecciona 10 enlaces aleatorios de la lista
# Este lo puedes ajustar como gustes, yo puse 10 para tener de donde escoger.
link_chambas = random.sample(link_chambas, min(len(link_chambas), 10))
link_chambas



https://www.glassdoor.com.mx/Empleo/estados-unidos-software-engineer-empleos-SRCH_IL.0,14_IN1_KO15,32.htm


['https://www.glassdoor.com/job-listing/software-engineer-mercor-JV_KO0,17_KE18,24.htm?jl=1009833586532',
 'https://www.glassdoor.com/job-listing/software-engineer-netheart-technology-JV_KO0,17_KE18,37.htm?jl=1009825160608',
 'https://www.glassdoor.com/job-listing/full-stack-software-engineer-frontend-focus-oneplan-solutions-JV_IC1139761_KO0,43_KE44,61.htm?jl=1009843374256',
 'https://www.glassdoor.com/job-listing/forward-deployed-engineer-software-engineering-initio-capital-JV_IC1147401_KO0,46_KE47,61.htm?jl=1009841907102',
 'https://www.glassdoor.com/job-listing/software-engineer-ii-omada-health-JV_IC1147401_KO0,20_KE21,33.htm?jl=1009839529494',
 'https://www.glassdoor.com/job-listing/software-engineer-reveleer-JV_KO0,17_KE18,26.htm?jl=1009843242257',
 'https://www.glassdoor.com/job-listing/software-engineer-coursedog-JV_KO0,17_KE18,27.htm?jl=1009841349691',
 'https://www.glassdoor.com/job-listing/software-engineer-supplyhouse-JV_KO0,17_KE18,29.htm?jl=1009836749826',
 'https://www.gl

## Scrapping de Trabajos y Exportación EXCEL

In [12]:
data = []

for url in link_chambas:

    service = Service(executable_path=ruta_driver)
    browser = webdriver.Chrome(service=service, options=chrome_options)
    browser.get(url)

    wait_time = random.uniform(3, 6)
    print(f"Esperando {wait_time:.2f} segundos antes de continuar...")
    time.sleep(wait_time)

    soup = bs(browser.page_source, 'html.parser')

    vacante = {}

    try:
        info = soup.find('div',{'class':'JobDetails_jobDetailsContainer__y9P3L'})
        vacante["Descripción"] = info.get_text(separator='\n').strip()
    except AttributeError:
        print(f"No se encontró la info en: {url}")

    vacante["Enlace"] = url  # Esto no genera error, así que no necesitas try/except aquí

    data.append(vacante)
    print(vacante)

    print("")
    browser.quit()

# Eliminar fragmento de texto especifico que siempre sale.
for vacante in data:
    vacante["Descripción"] = vacante["Descripción"].replace("Postularse en el sitio del empleador\nPostúlate ahora", "")

# Guardar datos como JSON estructurado y legible
nombre_archivo = f"GlassDoor_{busqueda.replace(' ', '_')}.json"
with open(nombre_archivo, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("Los datos se han exportado correctamente como JSON.")


Esperando 4.21 segundos antes de continuar...
{'Descripción': 'MERCOR\nSoftware Engineer\nInicia sesión para postularte\nRemote\nInicia sesión para postularte\nMercor is partnering with leading AI labs to recruit \nskilled Software Engineers\n for cutting-edge research initiatives. These projects focus on \nwriting and reviewing full unit test suites\n for complex open-source codebases across a variety of languages and frameworks.\nAbout the Role\nThis project involves:\nWriting and reviewing comprehensive unit tests for high-complexity code repositories.\nInterfacing directly with leading AI researchers from top-tier labs.\nWorking fully asynchronously, on your own schedule.\nA commitment of 15–40 hours/week depending on your availability and performance.\nYou’re a Strong Fit If You:\nHave 2+ years of experience (professional or open-source) in Java, TypeScript, JavaScript, Python or Go.\nHave hands-on experience writing unit tests in at least one of the above languages.\nHave contrib

# Open AI USE

In [13]:
class structure(BaseModel):
    place: str
    company: str
    position: str
    academic_profile: str
    english_level: str
    technical_skills: str
    soft_skills: str
    salary: str
    source: str
    link: str

In [ ]:
resultados = []
df = pd.DataFrame()
df_vacantes = pd.DataFrame(data)

columnas_estructuradas = structure.model_fields.keys()
for col in columnas_estructuradas:
    df_vacantes[col] = None

for idx, row in df_vacantes.iterrows():
    texto = row["Descripción"]
    enlace = row["Enlace"]
    print(f"Procesando vacante {idx+1} de {len(df_vacantes)}: {enlace}")


    try:
        response = client.responses.parse(
            model = "gpt-5",
            input = [
                {
                    #Instrucciones para el asistente
                    "role": "system",
                    "content": "Eres un asistente experto en extraer información de ofertas laborales. Tu tarea es analizar el texto proporcionado y generar una estructura con los campos requeridos sin alterar el texto original (A menos que se te especifique, como es el caso de English_level). No debes incluir ningún comentario o explicación adicional, solo el contenido estructurado con la información extraída. En el campo de 'english_level', si no se menciona explicitamente pero se infiere que el puesto requiere un nivel de inglés avanzado (por ejemplo, que el pais de la solicitud sea angloparlante), debes asignar 'Nativo'. Si no se menciona el nivel de inglés, asigna 'No especificado'. El apartado de 'source' siempre sera 'Glassdoor'.",
                },
                {
                    "role": "user",
                    "content": f"Usa el siguiente texto para extraer la información requerida:\n\n{texto}, enlace: {enlace}"
                },
            ],
            #temperature=0.2,
            reasoning={"effort": "minimal"},
            text_format=structure,
        )
        
        resultados = pd.DataFrame(response.output_parsed)
        resultados = resultados.transpose() # Transponer el DataFrame
        resultados.columns = resultados.iloc[0] # Columna

        resultados = resultados[1:]  # Eliminar la primera fila que ahora es el encabezado
        df = pd.concat([df, resultados], ignore_index=True)
        print("Trabajos formateados exitosamente.")
        df
    


    except Exception as e:
        print(f"Error al procesar la vacante {enlace}")
        print(e)


Procesando vacante 1 de 10: https://www.glassdoor.com/job-listing/software-engineer-mercor-JV_KO0,17_KE18,24.htm?jl=1009833586532
Trabajos formateados exitosamente.
Procesando vacante 2 de 10: https://www.glassdoor.com/job-listing/software-engineer-netheart-technology-JV_KO0,17_KE18,37.htm?jl=1009825160608
Trabajos formateados exitosamente.
Procesando vacante 3 de 10: https://www.glassdoor.com/job-listing/full-stack-software-engineer-frontend-focus-oneplan-solutions-JV_IC1139761_KO0,43_KE44,61.htm?jl=1009843374256
Trabajos formateados exitosamente.
Procesando vacante 4 de 10: https://www.glassdoor.com/job-listing/forward-deployed-engineer-software-engineering-initio-capital-JV_IC1147401_KO0,46_KE47,61.htm?jl=1009841907102
Trabajos formateados exitosamente.
Procesando vacante 5 de 10: https://www.glassdoor.com/job-listing/software-engineer-ii-omada-health-JV_IC1147401_KO0,20_KE21,33.htm?jl=1009839529494
Trabajos formateados exitosamente.
Procesando vacante 6 de 10: https://www.glassdoor

---

In [16]:
df

,place,company,position,academic_profile,english_level,technical_skills,soft_skills,salary,source,link
0,Remote,MERCOR,Software Engineer,,Nativo,Unit testing; Java; TypeScript; JavaScript; Py...,Interfacing directly with leading AI researche...,Pay: $50.00 - $85.00 per hour; Expected hours:...,Glassdoor,https://www.glassdoor.com/job-listing/software...
1,Remote,Netheart Technology,Software Engineer,No especificado,No especificado,"Yahoo finance API, data visualization, data in...",team leader prefer; flexible time; using slack...,no cash salary​ only have options plan until w...,Glassdoor,https://www.glassdoor.com/job-listing/software...
2,"Austin, TX",OnePlan Solutions,Full Stack Software Engineer (Frontend Focus),,Nativo,3-5 years in software development with JavaScr...,Strong communication skills; Collaboration wit...,,Glassdoor,https://www.glassdoor.com/job-listing/full-sta...
3,"San Francisco, CA",Initio Capital,Forward Deployed Engineer - Software Engineering,,Nativo,"Python, FastAPI, TypeScript, React; Backend de...",Customer-facing communication; Translate techn...,"$150,000 – $180,000 + Equity",Glassdoor,https://www.glassdoor.com/job-listing/forward-...
4,"San Francisco, CA",Omada Health,Software Engineer II,,Nativo,"3-5 years of experience writing readable, test...","You work with empathy for your coworkers, stak...","California, New York State and Washington Stat...",Glassdoor,https://www.glassdoor.com/job-listing/software...
5,Remote,Reveleer,Software Engineer / Software Developer / Java ...,"Bachelor's Degree in Computer Science, Compute...",Nativo,Core Java and J2EE/Enterprise Development (2 -...,Able to work independently and in a team envir...,Competitive pay (compensation reflects cost of...,Glassdoor,https://www.glassdoor.com/job-listing/software...
6,Remote,Coursedog,Software Engineer,No especificado,Nativo,3-5 years experience; Strong proficiency with ...,"innovative, fast-learning, detail-oriented; ow...",No especificado,Glassdoor,https://www.glassdoor.com/job-listing/software...
7,Remote from India,SupplyHouse,Software Engineer (Backend Engineer),Bachelor’s or Master’s degree in Computer Scie...,Nativo,3+ years of backend and/or full-stack developm...,Strong problem-solving and debugging capabilit...,"$25,000 - $30,000 USD per year",Glassdoor,https://www.glassdoor.com/job-listing/software...
8,Remote,"Sanguina, Inc.",Software Engineer - Backend,,No especificado,Backend API development supporting mobile appl...,Excellent communication skills; Entrepreneuria...,"From $140,000.00 per year",Glassdoor,https://www.glassdoor.com/job-listing/software...
9,"Boston, MA",Suno,"Software Engineer, Early Career",Bachelor's or Master's degree in Computer Scie...,Nativo,Strong foundation in computer science fundamen...,Strong problem-solving skills and attention to...,Compensation Range: $160K,Glassdoor,https://www.glassdoor.com/job-listing/software...


In [ ]:
# Exportar datos a Excel
df.to_excel(f"AI_PARSED_{busqueda.replace(' ', '_')}.xlsx", index=False, engine="openpyxl")